In [1]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasetlowestaic/dataset_lowestAIC.csv
/kaggle/input/iterated-12preds/iterated_12vars.csv
/kaggle/input/dataset-lowest-aic452022/dataset_lowestAIC_longlat.csv


In [2]:
df = pd.read_csv(r'../input/dataset-lowest-aic452022/dataset_lowestAIC_longlat.csv', sep=';')

In [3]:
#remove unique idenifier & geodata
df = df.drop(['Unnamed: 0', 'Longitude', 'Latitude'], axis=1)

In [4]:
#initialize response- and predictor variables
y = df['mean_value_NO2']
x = df.drop(['mean_value_NO2'], axis=1)
feature_list = list(x.columns)

In [5]:
                       ## CONSTRUCTING XGBoost
    ##IMPORT XGBOOST MODULE
                       ## Import the model we are using
        
import xgboost as xgb
from sklearn.model_selection import train_test_split


##STEP 8B. GENERATING CROSS VALIDATION (80% TRAINING OF MODEL; 
        ##20% TESTING OF MODEL;EVALUATING MODEL PERFORMANCE)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=20)
#random state is needed to ensure that same results are generated each time.

In [6]:
## ENSURING THAT X- AND Y-TRAINING SET CONTAIN SAME AMOUNT OF ROWS. SAME FOR TESTING.
print('Training Features Shape:',X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (361, 12)
Training Labels Shape: (361,)
Testing Features Shape: (121, 12)
Testing Labels Shape: (121,)


In [7]:
#construct model
xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
preds_test = xg_reg.predict(X_test)
preds_train = xg_reg.predict(X_train)

In [9]:
##RMSE

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print('RMSE training: ',rmse(preds_train, Y_train))
print('RMSE testing: ', rmse(preds_test, Y_test))

RMSE training:  0.0308734387690303
RMSE testing:  8.604244030335906


In [10]:
##R2

from sklearn.metrics import r2_score
print('R2 score training: ', r2_score(Y_train, preds_train))
print('R2 score testing: ', r2_score(Y_test, preds_test))

R2 score training:  0.9999939872662972
R2 score testing:  0.5954135512271237


In [11]:
##MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print('MAE training: ', mean_absolute_error(Y_train, preds_train))
print('MAE testing: ', mean_absolute_error(Y_test, preds_test))

MAE training:  0.0213100462577638
MAE testing:  6.288214168769282


In [12]:
#HYPERPARAMETER TUNING
#VERIFYING CURRENT RANDOM FOREST HYPERPARAMETERS 

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(xg_reg.get_params())

Parameters currently in use:

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 4,
 'num_parallel_tree': 1,
 'objective': 'reg:squarederror',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}


In [13]:
#Apply different parameter settings

xg_reg_adv = xgb.XGBRegressor(gamma = 5,  reg_alpha =2, reg_lambda=0, max_depth = 5, learning_rate = 0.002, n_estimators=2000, random_state=42)
xg_reg_adv.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=5, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.002, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=4, num_parallel_tree=1, random_state=42,
             reg_alpha=2, reg_lambda=0, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
##DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES
expected_y_H  = Y_test
predicted_y_adv = xg_reg_adv.predict(X_test)

expected_y_tr_H = Y_train
predicted_y_tr_adv = xg_reg_adv.predict(X_train)

In [15]:
##RMSE

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print('RMSE training: ',rmse(predicted_y_tr_adv, Y_train))
print('RMSE testing: ', rmse(predicted_y_adv, Y_test))

RMSE training:  2.187937877931452
RMSE testing:  8.565272808024908


In [16]:
##R2

from sklearn.metrics import r2_score
print('R2 score training: ', r2_score(Y_train, predicted_y_tr_adv))
print('R2 score testing: ', r2_score(Y_test, predicted_y_adv))

R2 score training:  0.9698024343913474
R2 score testing:  0.599070239975521


In [17]:
##MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print('MAE training: ', mean_absolute_error(Y_train, predicted_y_tr_adv))
print('MAE testing: ', mean_absolute_error(Y_test, predicted_y_adv))

MAE training:  1.609893907144724
MAE testing:  5.920493350545994
